### Librerias

In [69]:
import warnings
warnings.filterwarnings('ignore')
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import tensorflow as tf
from multiprocessing import Process
import threading


### funciones para preprocesar la img

In [ ]:
def redimensionar(img, nuevo_ancho, nuevo_alto):
    puntos_bajar = (nuevo_ancho, nuevo_alto)
    new_img = cv2.resize(img, puntos_bajar, interpolation= cv2.INTER_LINEAR)
    return new_img

In [52]:
def guardar_frame(frame):
    Datos='/home/lenin/Documents/GitHub/Modelo-para-clasificacion-de-colores-ANN/imgs/frame.jpg'
    cv2.imwrite(Datos, frame)

In [56]:
def dividir_img(img,nuevo_ancho, nuevo_alto, n_filas, n_column):
#     nuevo_ancho = 200
#     nuevo_alto = 200

    new_img = redimensionar(img, nuevo_ancho, nuevo_alto)
    # guardar_frame(new_img)
    img_base = new_img
    imgs=[]
    
    height, width, channels = new_img.shape
    CROP_W_SIZE  = n_filas
    CROP_H_SIZE = n_column

    for ih in range(CROP_H_SIZE ):
        for iw in range(CROP_W_SIZE ):

            x = width//CROP_W_SIZE * iw 
            y = height//CROP_H_SIZE * ih
            h = (height // CROP_H_SIZE)
            w = (width // CROP_W_SIZE )
            yy = int(y+h)
            xx = int(x+w)
            temporal = img_base[y:yy, x:xx]
            imgs.append(temporal)
            img_base = new_img
        
    return imgs

In [ ]:
def guardar_imgs(imgs):
    for i in range(len(imgs)):
        Datos='/home/lenin/Documents/GitHub/Modelo-para-clasificacion-de-colores-ANN/imgs/img{}.jpg'.format(i)
        cv2.imwrite(Datos, imgs[i])

In [ ]:
def predecir_imgs(modelo, imgs):
    clases = [] 
    vector = ['blanco', 'verde', 'gris', 'negro', 'azul', 'amarillo', 'cafe', 'naranja', 'rosado', 'morado', 'rojo', 'celeste']
    for i in range(len(imgs)):
        img = redimensionar(imgs[i],5,5)
        img = img[np.newaxis, ...]
        respuesta = modelo.predict(img, verbose=0)
        posicion = np.argmax(respuesta[0])
        if not vector[posicion] in clases:
            clases.append(vector[posicion])
    return clases

In [ ]:
# path = '/home/lenin/Pictures/fondo.jpg'
# img = load_img(path, target_size=(300,400))
# img = img_to_array(img)
# imgs = dividir_img(img, 200, 200, 5, 5)

In [ ]:
# modelo = tf.keras.models.load_model("mod_color_v1.h5")
# modelo.load_weights("mod_color_v1_weights.h5")
# clases = predecir_imgs(modelo, imgs)
# print(clases)

In [43]:
def adjust_brightness(img, alpha, beta):
    return cv2.addWeighted(img, alpha, np.zeros(img.shape, img.dtype), 0, beta)

### Preparo cv2

In [75]:
# 0 camara predeterminada
cap = cv2.VideoCapture(0)
imgs = []
clases = []
modelo = tf.keras.models.load_model("mod_color_v1.h5")
modelo.load_weights("mod_color_v1_weights.h5")
nuevo_size = 15
celda = nuevo_size // 5
crear_hilo = True
print(f'celda {celda}')

c=0
def procesamiento_paralelo(frame):
    try:
        imgs = dividir_img(frame, nuevo_size, nuevo_size, celda, celda)
        # guardar_imgs(imgs)
        clases = predecir_imgs(modelo, imgs)
        print(f"\rColores detectados: {clases}", end=" ")
    except Exception as e:
        print("Error en procesamiento paralelo:", e)
    
        
while True:
    # Capturar un marco
    ret, frame = cap.read()
    
    #volteo el frame
    frame = cv2.flip(frame, 1)
    frame = adjust_brightness(frame, 1.7, 0.1)
    
    # muestro la ventana
    cv2.imshow('frame', frame)

    # Salir si se presiona 'esc'
    key = cv2.waitKey(1)
    if key == 27:
        break
    # continue
    if c >= 30:
        c=0
        # procesamiento_paralelo(frame)
        # print(frame.shape)
        procesamiento_paralelo(frame)
        
    c+=1

cap.release()
cv2.destroyAllWindows()


celda 3
